In [ ]:
import os
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern

from advopt import *

SEED = 999111

In [ ]:
task = tasks.SwissRoll()

DATA_ROOT=os.path.join(
    os.environ.get('DATA_ROOT', './'),
    'RAO'
)

repeat = 100
budget = 256000

In [ ]:
results = experiment(
    DATA_ROOT,
    models=['JSD-GBDT', 'log-pJSD-GBDT', 'lin-pJSD-GBDT'],
    optimizer=bayesian_optimization(
        base_estimator=GaussianProcessRegressor(
            kernel=Matern(length_scale=1, length_scale_bounds=(1e-2, 1e+2)),
            alpha=1e-4
        ),
        n_initial_points=5
    ),
    task=task,
    budget=budget,
    repeat=repeat,
    progress=tqdm,
    devices=['cpu'],
    seed=SEED
)

In [ ]:
plt.figure(figsize=(6, 6))
plot_convergence(
    results, distance_to([0.0]), budget=budget, qs=(0.4, ),
    properties={
        'JSD-GBDT' : dict(color='black', label='JSD'),
        'lin-pJSD-GBDT' : dict(color=plt.cm.tab10(0), label='lin-pJSD'),
        'log-pJSD-GBDT' : dict(color=plt.cm.tab10(1), label='log-pJSD')
    },
    individual=False
)
plt.yscale('log')
plt.xlabel('examples sampled', fontsize=14)
plt.ylabel('distance to the solution', fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.savefig(os.path.join(
    DATA_ROOT, 
    '%s-%d-BO-GBDT.pdf' % (task.name(), budget)
))